This dataset contains information about 891 people who were on board the Titanic ship when it departed on April 15th, 1912. 
The following are the features of the dataset

- **Name** (str) - Name of the passenger
- **Pclass** (int) - Ticket class
- **Sex** (str) - Sex of the passenger
- **Age** (float) - Age in years
- **SibSp** (int) - Number of siblings and spouses aboard
- **Parch** (int) - Number of parents and children aboard
- **Ticket** (str) - Ticket number
- **Fare** (float) - Passenger fare
- **Cabin** (str) - Cabin number
- **Embarked** (str) - Port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)


In [27]:
#importing modules
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
# importing data
train_data = pd.read_csv("./titanic data/train.csv")

In [3]:
train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# checking if Ticket has some kind of grouping
train_data['Ticket'].value_counts()

1601        7
347082      7
CA. 2343    7
347088      6
3101295     6
           ..
345572      1
2625        1
315096      1
236853      1
A/5. 851    1
Name: Ticket, Length: 681, dtype: int64

In [6]:
# dropping columns that have no effect on the outcome of the model
train_data.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,NaN,S
887,1,1,female,19.0,0,0,30.0000,B42,S
888,0,3,female,NaN,1,2,23.4500,NaN,S
889,1,1,male,26.0,0,0,30.0000,C148,C


In [7]:
# checking whether missing values in age is random or systematic
train_data.groupby(train_data['Age'].isnull()).mean()

,Survived,Pclass,Age,SibSp,Parch,Fare
Age,,,,,,
False,0.406162,2.236695,29.699118,0.512605,0.431373,34.694514
True,0.293785,2.598870,NaN,0.564972,0.180791,22.158567


Although there are some significant values in Fare and Parch for when age was missing or not, the difference in survival rate for when age was missing or not is not so much. As a result, we can treat the missing values in age as random and not sytematic. Hence we can go ahead and fill in the missing values in age with the mean age, since doing so these not bias the model towards any outcome.

In [8]:
# filling in missing age values with mean age
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
train_data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

There are no longer missing values in the age column, but there are missing values in the Cabin and Embarked column

In [9]:
# checking if missing values in cabin in sytematic or random
train_data.groupby(train_data['Cabin'].isnull()).mean()

,Survived,Pclass,Age,SibSp,Parch,Fare
Cabin,,,,,,
False,0.666667,1.196078,35.258349,0.441176,0.436275,76.141504
True,0.299854,2.639010,28.048341,0.547307,0.365357,19.157325


Clearly, there is a higher survival rate(about 67%) for when Cabin values were not missing than when cabin values were missing(about 30%). This shows that the missing values in the Cabin column was systematic amd is a very strong indicator of whether someone survived or not.
As a result, although the missing values represent a very percentage or our dataset, we will not drop the missing values, but rather convert it to numerical values with a binary indicator to help our classification.

In [10]:
# creating a column called cabin_ind to store values for where cabin in null(0) or not(1)
train_data['Cabin_ind'] = np.where(train_data['Cabin'].isnull(), 0, 1)
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Cabin_ind
0,0,3,male,22.000000,1,0,7.2500,NaN,S,0
1,1,1,female,38.000000,1,0,71.2833,C85,C,1
2,1,3,female,26.000000,0,0,7.9250,NaN,S,0
3,1,1,female,35.000000,1,0,53.1000,C123,S,1
4,0,3,male,35.000000,0,0,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,NaN,S,0
887,1,1,female,19.000000,0,0,30.0000,B42,S,1
888,0,3,female,29.699118,1,2,23.4500,NaN,S,0
889,1,1,male,26.000000,0,0,30.0000,C148,C,1


In [11]:
# dropping cabin column
train_data.drop('Cabin', axis=1, inplace=True)
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin_ind
0,0,3,male,22.000000,1,0,7.2500,S,0
1,1,1,female,38.000000,1,0,71.2833,C,1
2,1,3,female,26.000000,0,0,7.9250,S,0
3,1,1,female,35.000000,1,0,53.1000,S,1
4,0,3,male,35.000000,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,0
887,1,1,female,19.000000,0,0,30.0000,S,1
888,0,3,female,29.699118,1,2,23.4500,S,0
889,1,1,male,26.000000,0,0,30.0000,C,1


In [12]:
# converting sex into numerical values
le = LabelEncoder()
train_data['Sex'] = le.fit_transform(train_data['Sex'])
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Cabin_ind
0,0,3,1,22.000000,1,0,7.2500,S,0
1,1,1,0,38.000000,1,0,71.2833,C,1
2,1,3,0,26.000000,0,0,7.9250,S,0
3,1,1,0,35.000000,1,0,53.1000,S,1
4,0,3,1,35.000000,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,S,0
887,1,1,0,19.000000,0,0,30.0000,S,1
888,0,3,0,29.699118,1,2,23.4500,S,0
889,1,1,1,26.000000,0,0,30.0000,C,1


In [13]:
# converting embarked column to numerical

dummies = pd.get_dummies(train_data['Embarked'], prefix='Embarked')
dummies


,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
886,0,0,1
887,0,0,1
888,0,0,1
889,1,0,0


In [14]:
train_data = pd.concat([train_data, dummies], axis=1)
train_data.drop('Embarked', inplace=True, axis=1)
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_ind,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.000000,1,0,7.2500,0,0,0,1
1,1,1,0,38.000000,1,0,71.2833,1,1,0,0
2,1,3,0,26.000000,0,0,7.9250,0,0,0,1
3,1,1,0,35.000000,1,0,53.1000,1,0,0,1
4,0,3,1,35.000000,0,0,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,0,0,0,1
887,1,1,0,19.000000,0,0,30.0000,1,0,0,1
888,0,3,0,29.699118,1,2,23.4500,0,0,0,1
889,1,1,1,26.000000,0,0,30.0000,1,1,0,0


In [18]:
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)


In [19]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [21]:
pred = clf.predict(X_test)
pred

array([1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1], dtype=int64)

In [29]:
report = classification_report(y_test, pred)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.84      0.82       110
           1       0.72      0.67      0.69        69

    accuracy                           0.77       179
   macro avg       0.76      0.75      0.75       179
weighted avg       0.77      0.77      0.77       179



In [25]:
matrix = confusion_matrix(y_test, pred)
matrix

array([[92, 18],
       [23, 46]], dtype=int64)

In [30]:
clf_1 = DecisionTreeClassifier(criterion='entropy')
clf_1.fit(X_train, y_train)
pred_1 = clf_1.predict(X_test)
report_1 = classification_report(y_test, pred_1)
print(report_1)

              precision    recall  f1-score   support

           0       0.79      0.83      0.81       110
           1       0.70      0.65      0.68        69

    accuracy                           0.76       179
   macro avg       0.75      0.74      0.74       179
weighted avg       0.76      0.76      0.76       179



Choosing entropy or gini does not really result in significant changes to accuracy, precision and recall.

In [38]:
clf_2 = DecisionTreeClassifier(criterion='gini', max_depth=7, min_samples_leaf=2, random_state=100)
clf_2.fit(X_train, y_train)
pred_2 = clf_2.predict(X_test)
report_2 = classification_report(y_test, pred_2)
print(report_2)

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       110
           1       0.80      0.62      0.70        69

    accuracy                           0.79       179
   macro avg       0.79      0.76      0.77       179
weighted avg       0.79      0.79      0.79       179

